In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Changes
alpha = 0.05
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [63]:
#Importing the data from GitHub
churn_raw  = pd.read_csv("https://raw.githubusercontent.com/johnson80245/python/refs/heads/main/churn_raw_data.csv")
churn = churn_raw.copy()

In [65]:
#Renaming the columns to correct survey response
churn = churn.rename( columns = {
    "CaseOrder" : "case_order",
    "TimeZone" : "time_zone",
    "InternetService" : "internet_service",
    "OnlineSecurity" : "online_security",
    "OnlineBackup" : "online_backup",
    "DeviceProtection" : "device_protections",
    "TechSupport" : "tech_support", 
    "StreamingTV" : "streaming_tv", 
    "StreamingMovies" : "streaming_movies",
    "PaperlessBilling" : "paperless_billing",
    "PaymentMethod" : "payment_method",
    "MonthlyCharge" : "monthly_charge",
    "item1" : "timely_response",
    "item2" : "timely_fixes",
    'item3' : "timely_replacements",
    'item4' : "reliability",
    "item5" : "options",
    "item6" : "respectful_responses",
    "item7" : "courteous_exchange",
    "item8" : "evidence_of_active_listening"
})

In [66]:
#Cleaning column names:
def to_clean(val):
    return val.strip().lower().replace(" ","_")

churn = churn.rename(columns = to_clean)

In [67]:
#Changing mapping of true/false  & yes/no

bool_map = {'False': 0, 'True': 1}
churn_map = {'Yes':1,'No':0}
bool_cols = ['techie', 'port_modem', 'tablet', 'phone', 'multiple',
           'online_security', 'online_backup', 'device_protections',
           'tech_support', 'streaming_tv', 'streaming_movies', 'paperless_billing']

for column in bool_cols:
    churn[bool_cols] = churn[bool_cols].replace(bool_map)

#churn[bool_cols] = churn[bool_cols].replace(bool_map)
churn['churn'] = churn['churn'].replace(churn_map)



C:\Users\Kyle\AppData\Local\Temp\ipykernel_7248\3242152760.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  churn['churn'] = churn['churn'].replace(churn_map)


In [69]:
#Cast as categorical data

cat_cols = ['case_order', 'customer_id', 'interaction', 'city', 'state', 'county',
            'area', 'timezone', 'job', 'education', 'employment',
            'marital', 'gender', 'churn', 'techie', 'contract',
            'port_modem', 'tablet', 'internet_service', 'phone', 'multiple',
            'online_security', 'online_backup', 'device_protections', 'tech_support', 'streaming_tv',
            'streaming_movies', 'paperless_billing', 'payment_method', 'timely_response', 'timely_fixes',
            'timely_replacements', 'reliability', 'options', 'respectful_responses',
            'courteous_exchange', 'evidence_of_active_listening']


for column in cat_cols:
    churn[column] = churn[column].astype('category')

In [60]:
#Finding the dtypes
churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 51 columns):
 #   Column                        Non-Null Count  Dtype   
---  ------                        --------------  -----   
 0   case_order                    10000 non-null  category
 1   customer_id                   10000 non-null  category
 2   interaction                   10000 non-null  category
 3   city                          10000 non-null  category
 4   state                         10000 non-null  category
 5   county                        10000 non-null  category
 6   zip                           10000 non-null  int64   
 7   lat                           10000 non-null  float64 
 8   lng                           10000 non-null  float64 
 9   population                    10000 non-null  int64   
 10  area                          10000 non-null  category
 11  timezone                      10000 non-null  category
 12  job                           10000 non-null  c

In [61]:
#Describe all transformed
churn.describe().T

,count,mean,std,min,25%,50%,75%,max
zip,10000.0,49153.319600,27532.196108,601.000000,26292.500000,48869.500000,71866.500000,99929.00000
lat,10000.0,38.757567,5.437389,17.966120,35.341828,39.395800,42.106908,70.64066
lng,10000.0,-90.782536,15.156142,-171.688150,-97.082812,-87.918800,-80.088745,-65.66785
population,10000.0,9756.562400,14432.698671,0.000000,738.000000,2910.500000,13168.000000,111850.00000
children,7505.0,2.095936,2.154758,0.000000,0.000000,1.000000,3.000000,10.00000
age,7525.0,53.275748,20.753928,18.000000,35.000000,53.000000,71.000000,89.00000
income,7510.0,39936.762226,28358.469482,740.660000,19285.522500,33186.785000,53472.395000,258900.70000
outage_sec_perweek,10000.0,11.452955,7.025921,-1.348571,8.054362,10.202896,12.487644,47.04928
email,10000.0,12.016000,3.025898,1.000000,10.000000,12.000000,14.000000,23.00000
contacts,10000.0,0.994200,0.988466,0.000000,0.000000,1.000000,2.000000,7.00000


In [ ]:
#Discovering missing values:
churn.isnull().sum()
#Missing values in:
    # -Children, Age, Income, techie, internet service, 
    # phone, tech support, tenure, bandwidth

#How to handle. 
#Income - Look at state, age, segments
#Children - Look at age group segments if older than x no kids if in range of y-z look at segments
    # Could also look at states children 
#Internet Service - if they stream tv or movies then true else false



def int_missing c




case_order                         0
customer_id                        0
interaction                        0
city                               0
state                              0
county                             0
zip                                0
lat                                0
lng                                0
population                         0
area                               0
timezone                           0
job                                0
children                        2495
age                             2475
education                          0
employment                         0
income                          2490
marital                            0
gender                             0
churn                              0
outage_sec_perweek                 0
email                              0
contacts                           0
yearly_equip_failure               0
techie                          2477
contract                           0
p